# Westeros Tutorial - Adding representation of renewables (part 3/3): Introducing `renewable_resource_constraints`

This tutorial, which demonstrates how to apply various model features to provide a more realistic representation of renewable energy integration in the energy system, is comprised of three parts. Previously, we introduced constraints on [`"firm capacity"`](https://docs.messageix.org/en/stable/model/MESSAGE/model_core.html?highlight=FIRM_CAPACITY_PROVISION#equation-firm-capacity-provision) and [`"flexible generation"`](https://docs.messageix.org/en/stable/model/MESSAGE/model_core.html?highlight=flexibility#equation-system-flexibility-constraint).

In the third part we will show you how to introduce renewable resource potentials.  Up until now, `"wind_ppl"` activity was unrestricted.  In order to reflect the fact that there are limited wind potentials within a given region and the fact that these differ in quality, we will add [`renewable_potentials`][1] and [`renewable_capacity_factors`][1] for wind.

[1]: https://docs.messageix.org/en/stable/model/MESSAGE/model_core.html?highlight=renewable#constraints-representing-renewable-integration

<img src='_static/renewable_resource_res.png' width='900'>

Further information can be found in [Sullivan et al., 2013](https://doi.org/10.1016/j.esr.2013.01.001).

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros scenario which adds emission taxes (`westeros_emissions_taxes.ipynb`) and solved it successfully

In [ ]:
import ixmp

mp = ixmp.Platform()

## Load existing and clone to new scenario
We load the existing scenario '*carbon_tax*' and clone it to a new scenario '*renewable_potential*' to which we will apply the `renewable_resource_constraints` constraint.

In [ ]:
import message_ix

model = "Westeros Electrified"
base = message_ix.Scenario(mp, model=model, scenario="carbon_tax")
scen = base.clone(
    model,
    "renewable_potential",
    "illustration of renewable_resource_constraint formulation",
    keep_solution=False,
)
scen.check_out()

## Retrieve parameters
We will retrieve those parameters necessary to perform subsequent additions of parameters.

In [ ]:
country = "Westeros"
year_df = scen.vintage_and_active_years((country, "wind_ppl"), in_horizon=False)
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")

## `renewable_resource_constraints` - Describing the renewable resource potentials
From the previous tutorials we know that in 720 wind capacity reaches over 150 GWa. We will therefore define 4 wind potential categories which in total will provide 200 GWa, yet the quality of these potentials will vary substantially from the current assumptions, where the capacity factor for `"wind_ppl"` has been assumed to be 1, meaning that the installed `"wind_ppl"` capacity can operate 8760 hours per year i.e., 100% of the year. Depending on the region, high quality on-shore wind potentials result in capacity factors around 35%, yet the majority of the potentials will lie below this value.  Therefore, 4 resource categories will be introduced:

| Resource Category | Potential \[GWa\] | Capacity Factor \[%\] |
| :---------------- | :---------------- | :-------------------- |
| c1 | 100 | 15 |
| c2 | 50 | 20 |
| c3 | 25 | 25 |
| c4 | 25 | 30 |

The figure below illustrates the potential categories as listed in the above table.

<img src='_static/westeros_renewable_resource_potentials.png' width='500'>

The capacity factor of the `"wind_ppl"` will remain unchanged and will be reflected in the parametrization of the renewable resources.  

The following steps are required:
1. Add level and commodity:
   - Specify a new level and commodity which account for the wind potentials and which serve as inputs to the `"wind_ppl"`
   - Specify which level is a `"level_renewable"`
2. Modify existing renewable technology:
   - Specify which technology is classified as a `"type_tec"` `"renewable"` (optional) 
   - Modify the input of the `"wind_ppl"`
3. Add potentials and corresponding capacity factors:
   - Add grades
   - Add `"renewable_potential"`s
   - Add `"renewable_capacity_factor"`s

### 1. Add level and commodity
The level and commodity which we add will allow us to account for potentials for wind.

In [ ]:
scen.add_set("level", ["renewable"])
scen.add_set("commodity", ["wind_onshore"])
scen.add_set("level_renewable", ["renewable"])

### 2. Modify existing renewable technology

#### Define a new technology category `"renewable"`
We will define `"type_tec"` and add `"wind_ppl"` to this new category. This can be used e.g. to simplify the reporting code, where results can be retrieved for technologies within given sets as opposed to specifying individual technologies.

In [ ]:
import pandas as pd

scen.add_set("type_tec", ["renewable"])
df = pd.DataFrame({"type_tec": ["renewable"], "technology": ["wind_ppl"]})
scen.add_set("cat_tec", df)

#### Add `"input"` parameter for `"wind_ppl"`
We will add the parameter `"input"` for `"wind_ppl"`.

In [ ]:
df = pd.DataFrame(
    {
        "node_loc": country,
        "technology": "wind_ppl",
        "year_vtg": vintage_years,
        "year_act": act_years,
        "mode": "standard",
        "node_origin": country,
        "commodity": "wind_onshore",
        "level": "renewable",
        "time": "year",
        "time_origin": "year",
        "value": 1,
        "unit": "%",
    }
)
scen.add_par("input", df)

### 3. Add potentials and corresponding capacity factors

#### Add new resource potential categories
Each renewable potential category is defined as a separate `"grade"`.

In [ ]:
grades = ["c1", "c2", "c3", "c4"]
scen.add_set("grade", grades)

#### Add `"renewable_potential"`s
Note that unlike fossil resources which are finite, renewable resources must be defined for each year.

In [ ]:
# `"renewable_potential"` has the following index structure
scen.idx_names("renewable_potential")

In [ ]:
idx = pd.MultiIndex.from_product(
    [[country], ["wind_onshore"], grades, ["renewable"], model_horizon, ["GWa"]],
    names=["node", "commodity", "grade", "level", "year", "unit"],
)
df = pd.DataFrame(
    {"value": sorted([100, 50, 25, 25] * len(model_horizon), reverse=True)}, idx
).reset_index()
scen.add_par("renewable_potential", df)

#### Add `"renewable_capacity_factor"`s

In [ ]:
# `"renewable_capacity_factor"` has the following index structure
scen.idx_names("renewable_capacity_factor")

In [ ]:
idx = pd.MultiIndex.from_product(
    [[country], ["wind_onshore"], grades, ["renewable"], model_horizon, ["-"]],
    names=["node", "commodity", "grade", "level", "year", "unit"],
)
df = pd.DataFrame(
    {"value": sorted([0.15, 0.20, 0.25, 0.30] * len(model_horizon))}, idx
).reset_index()
scen.add_par("renewable_capacity_factor", df)

## Commit and solve

In [ ]:
scen.commit(comment="Define parameters for renewable implementation")
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var("OBJ")["lvl"]

## Plotting Results

In [ ]:
from message_ix.report import Reporter
from message_ix.util.tutorial import prepare_plots

%matplotlib inline

rep_base = Reporter.from_scenario(base)
prepare_plots(rep_base)

rep_scen = Reporter.from_scenario(scen)
prepare_plots(rep_scen)

### Activity
***
When comparing the results of the original scenario without the renewable potentials ('*carbon_tax*') with the results of our newly modified scenario ('*renewable_potential*'), for the same carbon price we can observe that the activity of the `"wind_ppl"` has substantially decreased.  This is because through adding potentials with corresponding plant factors, the `"wind_ppl"` has become increasingly economically unattractive and despite the carbon tax is not used. 
Note that the `"coal_ppl"` still has a plant factor of 1 and has no resource constraints, thus in order to further improve the model, the parameters for the `"coal_ppl"` would need to be adjusted.

#### Scenario: '*carbon_tax*'

In [ ]:
rep_base.set_filters(t=["coal_ppl", "wind_ppl"])
rep_base.get("plot activity")

#### Scenario: '*renewable_potential*'

In [ ]:
rep_scen.set_filters(t=["coal_ppl", "wind_ppl"])
rep_scen.get("plot activity")

### Capacity
***
The behavior observed for the activity of the two electricity generating technologies is reflected in the capacity. No further capacity is built for the `"wind_ppl"` and thus it is phased out by 720.

#### Scenario: '*carbon_tax*'

In [ ]:
rep_base.get("plot capacity")

#### Scenario: '*renewable_potential*'

In [ ]:
rep_scen.get("plot capacity")

### Prices
***
Especially in the earlier model time periods, electricity and therefore the price for light increase dramatically.  The increase in 720 is due to the emission taxes associated with the operation of the `"coal_ppl"`.

#### Scenario: '*carbon_tax*'

In [ ]:
rep_base.set_filters(t=None, c=["light"])
rep_base.get("plot prices")

#### Scenario: '*renewable_potential*'

In [ ]:
rep_scen.set_filters(t=None, c=["light"])
rep_scen.get("plot prices")

In [ ]:
mp.close_db()

<div class="alert alert-block alert-success">
    
***Additional exercise***
    
The renewable potential categories have been defined such that `"capacity_factor"` decreases with increasing potentials. The model will thus first make of use the renewable potential with the highest capacity factor, and when saturated, the model proceeds with the next highest capacity factor. Typically, the potentials of better quality are not necessarily located close to the demand centers. As an exercise in a separate tutorial, add a cost to these potentials, by adding one technology for each grade, called something like "connection_to_grid_\<potential grade name\>", with variable costs as shown in the table below.
    
| Resource Category | Potential \[GWa\] | Capacity Factor \[%\] | Variable OM Cost in \[USD/kWa\] |
| :---------------- | :---------------- | :-------------------- | :------------------------------|
| c1 | 100 | 15 | 1 |
| c2 | 50 | 20 | 15 |
| c3 | 25 | 25 | 10 |
| c4 | 25 | 30 | 30 |
    
Remember that each of the renewable potential categories will require an individual `"commodity"` as input to the `"wind_ppl"`.
    
<img src='_static/renewable_resource_res_exercise.png' width='900'>